## First, get accuracy for persistance forecast and WG forecast

In [2]:
''' Import relevant libraries '''
import numpy as np
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
from numpy import array
from keras.callbacks import Callback
import keras.backend as K
import math
from sklearn.metrics import mean_squared_error

C:\Users\james\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Import all wind data after filtering for direction and speed

# cols are: [0 Date, 1 chimet speed, 2 chimet dir, 3 WG speed, 4 Wg dir]

data_SW10 = np.load('data_SW10.npy')


In [62]:
# Calculate WG rmse
WG_rmse = mean_squared_error(data_SW10[:,-12:,1],data_SW10[:,-12:,3])**(1/2)
print(WG_rmse)

6.195358878082382


In [61]:
# Creat persistance column
persistance=data_SW10
for i in range(0,data_SW10.shape[0]):
    persistance[i,:,2]=np.ones(48)*data_SW10[i,35,1]
    
np.delete(persistance,3,axis=2)
np.delete(persistance,4,axis=2)

prst_rmse = mean_squared_error(persistance[:,-12:,1],persistance[:,-12:,2])**(1/2)
print(prst_rmse)

5.5565203973061905


## Build LSTM

In [5]:
data_SW10.shape[1]

48

In [74]:
''' fit an LSTM network to training data '''
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons, dropout=0.5):
    
    # make training and validation set lengths divisble by batch num
    len_train = np.floor(train.shape[0]*0.5/n_batch)*n_batch*2
    len_train = len_train.astype(int)
    print(len_train)
    
    # reshape training into [samples, timesteps, features]
    X, y = train[:len_train, 0:n_lag], train[:len_train, n_lag:]
    X = X.reshape(X.shape[0], X.shape[1], 1)
    
    # design network
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True, recurrent_dropout=dropout))
    model.add(Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    # define restart schedule
    schedule = SGDRScheduler(min_lr=1e-5,
                        max_lr=1e-2,
                        steps_per_epoch=train.shape[0],
                        lr_decay=0.9,
                        cycle_length=5,
                        mult_factor=1.5)
    
    # fit network
    #model.fit(X, y, epochs=nb_epoch, batch_size=n_batch, shuffle=True, verbose=1, validation_split=0.5)
    model.fit(X, y, epochs=nb_epoch, batch_size=n_batch, shuffle=True, verbose=1, validation_split=0.5,callbacks=[schedule])
    #model.reset_states()
    
    return model

In [32]:
class SGDRScheduler(Callback):
    '''Cosine annealing learning rate scheduler with periodic restarts.

    # Usage
        ```python
            schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=1e-2,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.5)
            model.fit(X_train, Y_train, epochs=100, callbacks=[schedule])
        ```

    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        lr_decay: Reduce the max_lr after the completion of each cycle.
                  Ex. To reduce the max_lr by 20% after each cycle, set this value to 0.8.
        cycle_length: Initial number of epochs in a cycle.
        mult_factor: Scale epochs_to_restart after each full cycle completion.

    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: http://arxiv.org/abs/1608.03983
    '''
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)

In [76]:
data_SW10.shape

(16721, 48, 5)

In [77]:
n_lag = 36
n_seq = 12
nb_epoch = 50
n_batch = 32   # batch size must be divisible by number of cores (8 in this case)
n_neurons = 32
dropout=0.8

train=data_SW10[:10816,:,1] #first half of data
test1=data_SW10[5408:16224,:,1]

lstm_fit=fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons, dropout)

10816
Train on 5408 samples, validate on 5408 samples
Epoch 1/50
5408/5408 [==============================] - 3s 550us/step - loss: 49.6350 - val_loss: 26.6993
Epoch 2/50
5408/5408 [==============================] - 3s 467us/step - loss: 27.9764 - val_loss: 25.2777
Epoch 3/50
5408/5408 [==============================] - 3s 618us/step - loss: 26.9625 - val_loss: 24.7937
Epoch 4/50
5408/5408 [==============================] - 3s 541us/step - loss: 26.5604 - val_loss: 24.1459
Epoch 5/50
5408/5408 [==============================] - 3s 565us/step - loss: 25.8308 - val_loss: 24.4242
Epoch 6/50
5408/5408 [==============================] - 2s 387us/step - loss: 26.2293 - val_loss: 24.4488
Epoch 7/50
5408/5408 [==============================] - 3s 615us/step - loss: 25.4486 - val_loss: 23.8910
Epoch 8/50
5408/5408 [==============================] - 3s 602us/step - loss: 25.3515 - val_loss: 23.9727
Epoch 9/50
5408/5408 [==============================] - 3s 610us/step - loss: 25.2164 - val_loss: 

In [28]:
lstm_pred = Sequential()
lstm_pred.add(LSTM(n_neurons, batch_input_shape=(2, 36, 1), stateful=True, recurrent_dropout=dropout))
lstm_pred.add(Dense(12))
lstm_pred.compile(loss='mean_squared_error', optimizer='adam')

# copy weights
old_weights = lstm_fit.get_weights()
lstm_pred.set_weights(old_weights)

X = test1[:2, 0:n_lag]
X = X.reshape(X.shape[0], X.shape[1], 1)
X.shape
lstm_pred.predict(X)

array([[14.242188, 14.056882, 13.916418, 13.770002, 13.595895, 13.643635,
        13.385409, 13.424304, 13.511431, 13.423963, 13.295616, 13.375128],
       [11.792924, 11.826064, 11.863474, 11.862047, 11.84593 , 12.036204,
        11.933744, 12.090574, 12.27532 , 12.267679, 12.238291, 12.379164]],
      dtype=float32)

In [78]:
for i in range(0,338):
    X = test1[i*32:(i+1)*32, 0:n_lag]
    X = X.reshape(X.shape[0], X.shape[1], 1)
    pred=lstm_fit.predict(X)
    if i == 0:
        pred1=pred
    else:
        pred1=np.append(pred1,pred,axis=0)

X=pred1

In [79]:
X.shape

(10816, 12)

In [80]:
X1 = X.reshape(X.shape[0], X.shape[1], 1)
X2 = data_SW10[5408:16224,-12:,3]
X2 = X2.reshape(X2.shape[0], X2.shape[1], 1)
X2.shape
X=np.append(X1,X2,axis=2)

In [86]:
y=data_SW10[5408:16224,-12:,1]

nb_epoch = 200
n_batch = 32   # batch size must be divisible by number of cores (8 in this case)
n_neurons = 16
dropout=0.8

#create level 2 lstm
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True, recurrent_dropout=dropout))
model.add(Dense(y.shape[1]))
model.compile(loss='mean_squared_error', optimizer='adam')

schedule = SGDRScheduler(min_lr=1e-5,
                    max_lr=1e-2,
                    steps_per_epoch=train.shape[0],
                    lr_decay=0.9,
                    cycle_length=5,
                    mult_factor=1.5)

''' Fit model '''
model.fit(X, y, epochs=nb_epoch, batch_size=n_batch, shuffle=True, verbose=1, validation_split=0.5,callbacks=[schedule],)

Train on 5408 samples, validate on 5408 samples
Epoch 1/200
5408/5408 [==============================] - 2s 343us/step - loss: 79.4114 - val_loss: 42.1839
Epoch 2/200
5408/5408 [==============================] - 1s 221us/step - loss: 35.5355 - val_loss: 35.5839
Epoch 3/200
5408/5408 [==============================] - 1s 222us/step - loss: 33.0881 - val_loss: 30.4781
Epoch 4/200
5408/5408 [==============================] - 1s 229us/step - loss: 30.7409 - val_loss: 27.3576
Epoch 5/200
5408/5408 [==============================] - 1s 249us/step - loss: 27.5808 - val_loss: 24.7455
Epoch 6/200
5408/5408 [==============================] - 1s 240us/step - loss: 25.2512 - val_loss: 22.9481
Epoch 7/200
5408/5408 [==============================] - 1s 251us/step - loss: 24.0379 - val_loss: 23.1218
Epoch 8/200
5408/5408 [==============================] - 1s 242us/step - loss: 23.0339 - val_loss: 21.4918
Epoch 9/200
5408/5408 [==============================] - 1s 243us/step - loss: 22.2991 - val_los

5408/5408 [==============================] - 1s 229us/step - loss: 21.0382 - val_loss: 20.7834
Epoch 152/200
5408/5408 [==============================] - 1s 245us/step - loss: 21.0527 - val_loss: 21.8233
Epoch 153/200
5408/5408 [==============================] - 1s 236us/step - loss: 21.0652 - val_loss: 21.9658
Epoch 154/200
5408/5408 [==============================] - 1s 237us/step - loss: 21.0209 - val_loss: 21.4503
Epoch 155/200
5408/5408 [==============================] - 1s 246us/step - loss: 20.9953 - val_loss: 20.3531
Epoch 156/200
5408/5408 [==============================] - 1s 246us/step - loss: 21.0617 - val_loss: 20.4070
Epoch 157/200
5408/5408 [==============================] - 1s 242us/step - loss: 21.0441 - val_loss: 21.0707
Epoch 158/200
5408/5408 [==============================] - 1s 235us/step - loss: 21.1128 - val_loss: 20.6594
Epoch 159/200
5408/5408 [==============================] - 1s 241us/step - loss: 21.1783 - val_loss: 20.7102
Epoch 160/200
5408/5408 [========

In [87]:
print(20.2675**0.5)

4.501944024529847


In [36]:
from keras.layers import Concatenate

X1 = data_SW10[:16224,:,1]
X1 = X1.reshape(X1.shape[0], X1.shape[1], 1)
X2 = data_SW10[:16224,-12:,3]
X2 = X2.reshape(X2.shape[0], X2.shape[1], 1)
y  = data_SW10[:16224,-12:,1]

n_batch = 32
dropout = 0.5
n_neurons = 8
nb_epoch = 2

# design branch 1 (chimet hsit forecast)
branch1 = Sequential()
branch1.add(LSTM(n_neurons, batch_input_shape=(n_batch, X1.shape[1], X1.shape[2]), stateful=True, recurrent_dropout=dropout))
branch1.add(Dense(y.shape[1]))

# design branch 2 (windguru original forecast)
branch2 = Sequential()
branch2.add(LSTM(n_neurons, batch_input_shape=(n_batch, X2.shape[1], X2.shape[2]), stateful=True, recurrent_dropout=dropout))
branch2.add(Dense(y.shape[1]))

In [29]:
branch2.output

<tf.Tensor 'dense_25/BiasAdd:0' shape=(32, 12) dtype=float32>

In [38]:
model = Sequential()
#keras.layers.concatenate([branch1.output,branch2.output], axis=1)
model.add(keras.layers.concatenate([branch1.output,branch2.output], axis=1))
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, 24,1), stateful=True, recurrent_dropout=dropout))
model.compile(loss='mean_squared_error', optimizer='adam')
    
# define restart schedule
schedule = SGDRScheduler(min_lr=1e-5,
                    max_lr=1e-2,
                    steps_per_epoch=X1.shape[0],
                    lr_decay=0.9,
                    cycle_length=5,
                    mult_factor=1.5)

# fit network

model.fit([X1,X2], y, epochs=nb_epoch, batch_size=n_batch, shuffle=True, verbose=1, validation_split=0.5, callbacks=[schedule])

TypeError: The added layer must be an instance of class Layer. Found: Tensor("concatenate_16/concat:0", shape=(32, 24), dtype=float32)

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

b1_input = Input(shape=(100,), name='b1_input')